In [1]:
!pip install -U bitsandbytes transformers accelerate

from huggingface_hub import notebook_login
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import transformers
from tqdm import tqdm
from torch.utils.data import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 106.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import tqdm

# Check if GPU is available and set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Model and tokenizer configurations
model_name = "ruslanmv/Medical-Llama3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Define the query_model function to generate short answers
def query_model(prompt, temperature=0.0, max_length=50):
    # Adjust the prompt to encourage short answers, particularly for multiple-choice questions
    prompt = f"Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.\n\n{prompt}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=temperature,
        do_sample=False,  # Deterministic answers
        pad_token_id=tokenizer.pad_token_id,
    )
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    return response_text

# Load the CSV dataset
df = pd.read_csv('DiseasePredictor_test.csv')

# Initialize an empty list to store the generated answers
generated_answers = []

# Loop through each row in the dataset
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    try:
        question = row['Question']
        # Query the model with the current question
        response = query_model(question, temperature=0.0)
        print(response)
        generated_answers.append(response)
    except Exception as e:
        print(f"An error occurred for row {index}: {e}")
        generated_answers.append(None)  # Append None for failed generations

# Add the generated answers to the dataframe
df['MedicalLlama3_8B_Answer'] = generated_answers

# Save the dataframe with the answers to a new CSV file
df.to_csv('DiseaseDecipher_with_Answers_medicalllama.csv', index=False)

print("Model answers have been saved to 'DiseaseDecipher_with_Answers.csv'.")


Using GPU


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Central atherosclerosis 2) Otosclerosis 3) Irritable bowel syndrome 4) Cytomegalovirus infection
Answer: 2
Explanation: The patient is presenting with symptoms of otosclerosis, which is a condition where the bones of the middle ear become stiff and unable to vibrate properly. This can cause hearing loss, dizziness, and ear pain. The


100%|█████████▉| 3199/3200 [2:37:49<00:02,  2.93s/it]

Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Cytomegalovirus infection 2) Irritable bowel syndrome 3) Otosclerosis 4) Central atherosclerosis
Answer: 3
Explanation: The patient is presenting with symptoms of otosclerosis, which is a condition where the bones of the middle ear become stiff and unable to vibrate properly. This can cause hearing loss, dizziness, and ear pain. The


100%|██████████| 3200/3200 [2:37:52<00:00,  2.96s/it]

Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.

A patient is presenting with the following symptoms: Diminished hearing, Dizziness, Plugged feeling in ear, Ear pain, Elbow weakness, Nailbiting. Based on these symptoms, which of the following diseases is the most likely diagnosis?
1) Central atherosclerosis 2) Cytomegalovirus infection 3) Irritable bowel syndrome 4) Otosclerosis
Answer: 2
Explanation: The patient is presenting with symptoms of otitis media, which is an inflammation of the middle ear. The most likely diagnosis is cytomegalovirus infection, as this virus is known to cause otitis media. The other
Model answers have been saved to 'DiseaseDecipher_with_Answers.csv'.
